In [6]:
import torch
import numpy as np
import imageio
import os

# Chap 4

In [ ]:
# 비디오나 텍스트 같은 데이터를 텐서로 표현하여 딥러닝 모델 훈련에 적합하도록 가공하는 법

## 1절 - 이미지 다루기

In [ ]:
# 컨볼루션 신경망으로 이미지 기반 시스템에 가능성 열림
# 이미지 포맷을 읽어서 파이토치의 방식에 맞춰 이미지의 정보를 담은 텐서 표현으로 데이터 변환 필요

In [ ]:
# 이미지는 픽셀 단위의 높이와 너비를 가지는 표준적인 그리드에 나열된 복수 개의 스칼라 값 모음
# 여러 색을 표현하는 복수의 스칼라 값 및 심도 등의 피처를 포함하기도 함

In [ ]:
# 일반 카메라는 8비트 정수 인코딩, 의학 및 과학 및 군용 앱에서는 12 or 16비트
# 정밀도 높이면 물리적인 속성 정보 인코딩 할 때 더 넓은 범위나 높은 민감도 가지게 할 수 있음

In [ ]:
# 컬러를 숫자로 인코딩 하는 방법은 다양함
# 제일 흔한 RGB

In [ ]:
# imageio 모듈 사용한 PNG 이미지 로딩
import imageio

img_arr = imageio.imread('../data/bobby.jpg')
img_arr.shape

<ipython-input-8-0b45696610bd>:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('../data/bobby.jpg')


FileNotFoundError: No such file: '/data/bobby.jpg'

In [ ]:
# 너비와 높이에 해당하는 두 개의 공간 정보와 RGB에 해당하는 세 번째 차원으로 3차원 배열 같은 넘파이 객체
# 넘파이 배열 리턴하는 라이브러리면 텐서를 얻을 수 있으며 각 차원의 레이아웃만 조심할 것
# 이미지 다루는 파이토치 모듈은 텐서가 C*H*W(채널*높이*너비) 순으로 배치되야 함

In [ ]:
# permute 메소드로 채널 순서를 올바르게 변경
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

NameError: name 'img_arr' is not defined

In [ ]:
# 위 연산은 텐서 복사본을 만들지 않음
# out과 img 저장 공간은 동일 하며 텐서 레벨에서 크기와 스트라이드 정보만 변경되어 비용이 저렴한 연산
# img 픽셀 변경하는 경우 out에 있는 픽셀도 바뀜

In [ ]:
# 딥러닝 프레임워크(텐서플로 등)마다 레이아웃이 다를 수 있음

In [ ]:
# 첫 번째 차원에 여러 이미지를 batch로 넣어 N*C*H*W 텐서로 저장
# stack을 사용해 미리 공간을 할당하고 디렉토리에서 읽은 이미지로 채우기
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype = torch.int8) # 높이256, 너비 256, RGB 이미지 세 개로 이뤄진 batch / 각 색상이 8비트 정수로 표현될 것을 기대 -> dtype=torch.unit8 오류나서 torch.int8으로 변경

In [ ]:
data_dir = "../data/image-cats/"
filenames = [name for name in os.listdir(data_dir)
             if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3] # 투명도 알파 채널은 쓰지 않으므로 RGB있는 세 개 채널만 사용
    batch[i] = img_t

In [ ]:
# 신경망은 입력값이 0~1이거나 -1~1 일 때 훈련 성능이 가장 좋으므로 텐서를 부동소수점으로 캐스팅하고 정규화 진행
batch = batch.float() # 부동소수점으로 캐스팅
batch /= 255.0 # 정규화(원래 픽셀 값이 int8이었으므로 255로 나누기)

In [ ]:
# 입력 데이터의 평균과 표준 편차를 구해서 평균이 0이고 각 채널값이 표준 편차를 넘지 않게 만드는 방법
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

In [ ]:
# 입력에 대하여 정규화 외에 회전이나 크기 변환, 자르기 같은 추가적인 기하학적인 변화도 있음
# 이 작업들이 훈련에 도움이 될 수도 있고 신경망이 요구하는 입력 형태를 위한 필요 작업이 될 수도 있음

## 2절 - 3차원 이미지: 용적 데이터

In [ ]:
# CT는 흑백 이미지처럼 하나의 밀도 채널만 있음 즉 다른 색상 채널은 그대로 둔 채 데이터를 저장하기도 함 따라서 통상 세 개 채널이 다 들어 있음
# 2차원 단면을 스택처럼 쌓아 3차원 텐서를 만들어 3차원 해부도를 표현한 용적 데이터를 만들 수 있음
# CT 사진의 추가 차원은 물리적인 오프셋 정보(머리 혹은 발끝에서의 실제 거리)를 나타냄
# 용적 데이터는 채널 차원 뒤에 깊이 차원을 가지므로 5차원 텐서임

In [ ]:
# imageio모듈의 volread 함수로 디지털영상파일을 3차원 넘파이 배열로 만듦
dir_path = '../data/volumetric-dicom'
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

In [ ]:
# 채널 정보가 사라져서 파이토치에서 예상하는 레이아웃과 달라져 unsqueeze로 채널 차원 공간 생성
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)
vol.shape

## 3절 - 테이블 데이터 표현하기

In [ ]:
# 가장 단순한 형태의 데이터는 스프레드시트, CSV파일, 데이터베이스의 데이터
# 공통적으로 샘플 혹은 레코드별로 한 행에 해당하는 데이터이고 각 열은 샘플에 대한 정보 한 조각

In [ ]:
# 테이블 샘플의 순서가 의미 없다고 가정
# 시간 순으로 정렬된 시계열 데이터와 달리 독립적으로 존재한다고 생각
# 각 열의 타입이 동일하지 않음 ex) 사과의 무게, 사과의 색상 등
# 텐서의 내부 값은 형태가 float로 동일

In [ ]:
# 각자 다른 형태의 값인 실세계 데이터를 float로 이뤄진 텐서로 만들어 신경망에 넣을 준비

In [ ]:
# CSV파일 읽는 제일 흔한 방법: 파이썬 내장 csv 모듈 사용, 넘파이, 판다스
# 판다스 사용하는 방식이 제일 빠르고 메모리 절약됨

In [ ]:
# 파일 읽고 넘파이 배열 텐서로 변경
import csv
wine_path = '../data/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [ ]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [ ]:
wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [ ]:
# 품질 점수를 연속값으로 취급하여 실수 형태 유지한 채로 회귀 작업 수행할 수도 있고
# 레이블로 취급해서 분류 작업에서 화학 정보를 분석해 레이블 예측할 수도 있음
# 두 경우 모두 점수 텐서를 제거하여 독자적으로 유지하고 정답 값으로 사용 가능

In [ ]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [ ]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [ ]:
# 카테고리 데이터의 사용 용도에 따른 target 텐서 전치하는 두 가지 방법

In [ ]:
# 1.정수 벡터로 레이블 처리 / '와인 색'과 같이 문자열로 이루어진 경우 각 문자열마다 대응하는 정수 할당
target = wineq[:, -1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In [ ]:
# 2. 원핫 인코딩: 1~10의 값이 벡터 안의 10개의 원소에 대응하도록 정하고 원소 하나만 1로 설정하는 방식
# 점수 상에서 순서와 거리를 가정하면 정수 벡터 처리, 점수가 이산적이면 원핫 인코딩

In [ ]:
# scatter_ 메소드를 사용하여 소스 텐서와 함께 전달된 인자의 인덱스를 따라 새 텐서 채우기
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0) # scatter_(dim, index, src) dim: 작업을 수행할 차원 index: 작업을 수행할 위치를 나타내는 텐서 src: 할당될 값이 있는 텐서

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
# target 을 언스퀴즈하는 이유: 차원 늘릴려고
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [ ]:
# 데이터 가공
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [ ]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [ ]:
# 정규화
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [ ]:
# 점수 3 이하인 열 target에서 거르기
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [ ]:
# 파이토치의 고급 인덱싱 기능 이용하여 true 해당하는 행들만 접근
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [ ]:
# 위처럼 좋음, 보통, 나쁨 카테고리 나누기
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [ ]:
# 나쁜 와인의 경우 sulfur dioxide 성분이 높은 것으로 보이므로 total sulfor dioxide를 임계값으로 사용하여 중앙점보다 낮은 인덱스만 가져오기
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [ ]:
# 위 임계값으로는 전체 중 절반 이상을 고품질로 구별
# 실제로 좋은 와인의 인덱스 뽑기
actual_indexes = target > 5

actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [ ]:
# 임계값을 통한 예측 수량보다 500개 넘게 좋은 와인 존재하므로 완벽하지 않음
# 실제 순위와 우리가 예측한 순서가 얼마나 잘 맞는지 확인
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)

## 4절 - 시계열 데이터 다루기

In [ ]:
# 각 행은 연속적인 시간상에서의 한 지점을 표현
# 순서대로 나열 되며 만들어진 차원이 존재
# 각기 다른 값에 대해 연속으로 여러 개의 값을 볼 필요가 있음 -> 크기가 C인 N개의 병렬 시퀀스로 표현 가능

In [ ]:
bikes_numpy = np.loadtxt("../data/hour-fixed.csv", dtype=np.float32, delimiter=",", skiprows=1, converters={1: lambda x: float(x[8:10])})
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [ ]:
# 일 단위와 같은 다소 넓은 관찰 주기로 나누기 -> 길이가 L인 C개의 시퀀스를 가지는 N개의 컬렉션 얻어짐
# N*C*L인 3차원 텐서: C = 17개 채널, L = 24 1을 1시간으로 하여 24시간, N = 샘플 수
# 일별로 매 시간의 데이터 셋을 구하기 위해 동일 텐서를 24시간 배치로 보는 뷰 필요

In [ ]:
# bikes의 쉐이프와 스트라이드 확인
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [ ]:
# 데이터를 일자, 시간, 17개 열의 세 개 축으로 변경
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [ ]:
# C개 채널을 가진 하루를 L시간으로 나눈 N개의 연속된 값을 가질 때, N*C*L 순서로 놓기 위해 전치
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [ ]:
# 데이터 렌더링을 쉽게 하기 위하여 하루 시간대 만큼의 행과 날씨 상태 가짓수 만큼의 열을 가진 0으로 채워진 행렬을 초기화
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:, 9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [ ]:
# 날씨 수준에 따라 원핫인코딩으로 변환
weather_onehot.scatter_(dim=1, index=first_day[:,9].unsqueeze(1).long() - 1, value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [ ]:
# cat 함수를 사용해 원래 데이터셋에 만든 행렬을 병합
torch.cat((bikes[:24], weather_onehot), 1)[:1] # 17열 + 원핫인코딩 4열 => 21열 / torch.cat(tensors, dim=0, *, out=None) tensors: 연결할 텐서들을 나타내는 시퀀스(예: 리스트) dim: 연결할 차원 out: 결과를 저장할 텐서. 기본값은 None이며, 새로운 텐서가 반환

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [ ]:
# daily_bikes 텐서에도 동일 작업 (B, C, L) 형태이고 L=24임
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [ ]:
# 텐서의 C 차원에 원핫 인코딩 늘어놓기 텐서 안에서 수행하는 연산이므로 텐서의 내용이 변함
daily_weather_onehot.scatter_(1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [ ]:
# C차원 따라 원핫 인코딩 병합
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [ ]:
# 위 방법이 날씨 상태 값 처리하는 유일한 방법은 아님
# 레이블 자체에 순서 관계가 있으므로 연속 변수에 해당하는 특수값으로 가장할 수도 있음
daily_bikes[:,9,:] = (daily_bikes[:,9,:] - 1.0) / 3.0

In [ ]:
# 마찬가지로 변수를 [0.0, 1.0] or [-1.0, 1.0] 사잇값으로 변경하는 것은 모든 정량적인 변수 다루는 방법 ex) 10열의 temperature -> 이 방식은 훈련 과정에 도움을 줌

In [ ]:
# 값의 범위를 [0.0, 1.0]으로 매핑하는 방법
temp = daily_bikes[:,10,:]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:,10,:] = ((daily_bikes[:,10,:] - temp_min) / (temp_max - temp_min))

In [ ]:
# 모든 값에서 평균 빼고 표준편차로 나누는 방법
temp = daily_bikes[:,10,:]
daily_bikes[:,10,:] = ((daily_bikes[:,10,:] - torch.mean(temp)) / torch.std(temp)) # 평균 0, 단위 표준 편차를 가짐 이 경우는 샘플의 68%가 [-1.0, 1.0]에 분포

## 5절 - 텍스트 표현하기

In [ ]:
# 모델의 이전 출려과 현재의 입력을 섞는 식으로 반복해 소비하는 형태의 모델인 RNN(순환 신경망)
# RNN을 통한 자연어 처리 분야 해결
# 최근에는 transformer로 불리는 신경망으로 과거의 정보를 포함하는 유연한 방법을 통해 성공 중

In [ ]:
# 5절에서의 목표는 텍스트를 이전처럼 신경망이 처리할 수 있는 숫자의 텐서로 바꾸는 것
# 데이터의 차원 정보를 재정의 하는 것으로 시작

In [ ]:
# 텍스트를 다루는 직관적인 두 가지 방법
# 1. 문자 단위로 한 번에 하나의 문자 처리
# 2. 단어 단위로 신경망이 바라보는 세밀한 엔티티로 개별 단어 처리
# 둘 다 원핫 인코딩

In [ ]:
with open('../data/1342-0.txt', encoding='utf8') as f:
    text = f.read()

In [ ]:
# 한 줄 가져오기
lines = text.split('\n')
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [ ]:
# 첫 번째 방법
# 행 전체의 문자 원핫 인코딩한 문자의 총 수를 담을 텐서 제작
letter_t = torch.zeros(len(line), 128) # 아스키 제한인 128로 하드코딩
letter_t.shape

torch.Size([70, 128])

In [ ]:
# 각 행이 일치하는 문자 표현하도록 정확한 위치에 1 기록
# 값이 1이어야 할 인덱스는 인코딩에서의 문자 인덱스에 대응
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0 # 아스키에 유효하지 않은 문자 버림
    letter_t[i][letter_index] = 1

In [ ]:
# 두 번째 방법
# 출현된 단어로 사전을 만들어 문자에 나오는 단어 시퀀스에 대해 한 단어를 한 행으로 원핫 인코딩
# 사전에 단어가 매우 많아 인코딩 벡터가 매우 길어지면 실용성 없어짐

In [ ]:
# 텍스트 받아 소문자로 바꾸고 구두점 날리는 clean_words 정의
def clean_words(input_str):
    punctuation = '.,;:“!?`”_-' # 책에 있는 문자 못 찾아서 차이가 있음
    word_list = input_str.lower().replace('\n',' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [ ]:
# 인코딩에서 단어를 인덱스로 매핑
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

len(word2index_dict), word2index_dict['impossible']

(7261, 3394)

In [ ]:
# word2index_dict는 단어를 키로, 정수를 값으로 가지는 dictionary
# 인덱스를 효율적으로 찾을 용도
# 문장을 단어로 나누고 각각 원핫 인코딩하면 단어 하나당 원핫 인코딩된 벡터가 모여 텐서 하나 만듦
# 빈 벡터 만들고 각 단어의 원핫 인코딩 값 부여
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))

print(word_t.shape)

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
torch.Size([11, 7261])


In [ ]:
# 각자의 장단점
# 대부분의 언어는 단어 수보다 문자 수가 훨씬 적음
# 문자를 표현해 사용하면 표현 가능한 클래스도 적음
# 단어를 표현하면 매우 큰 수의 클래스 표현 가능하며 실제에서는 사전에 없는 단어도 다루게 됨
# 단어 표현은 문자보다 더 많은 의미를 내포하므로 자체적으로 훨씬 많은 정보를 가짐

In [ ]:
# 둘을 보완하는 방식인 바이트 쌍 인코딩(byte pair encoding)     # 이해 못 함
# 사전에 개별 문자 넣고 시작해 지정된 사전 크기에 도달할 때까지 가장 많이 발견된 쌍을 되풀이하여 사전에 넣는 식
# lines[200] 넣으면 다음과 같이 나옴
# ?Im|pos|s|ible|,|?Mr|.|?B|en|net|,|?impossible|,|?when|?I|?am|?not|?acquainted|?with|?him

In [ ]:
# 대부분의 매핑은 단어 단위로 구분하지만 대문자로 시작하는 경우에는 세부 단위 나누기도 함

In [ ]:
# 말뭉치의 단어에 해당하는 인코딩할 아이템 수를 효과적으로 제한 못하면 실패
# 중복된 단어를 제거하고 철자를 대체하여 가짓수 줄이거나, 과거 시제나 미래 시제를 구분하지 않고 하나의 토큰으로 보는 방법
# 새로운 단어 생기면 벡터에 열 추가하여 가중치 조정해야 함 -> 성가심

In [ ]:
# 부동소수점 수를 가지는 벡터로 개별 단어를 100차원 공간에 매핑해 학습을 가능하게 하는 임베딩(embedding)
# 이상적인 방법은 비슷한 맥락에서 사용된 단어들이 가까운 거리에 배치되는 임베딩

In [ ]:
# 임베딩 벡터에는 100~100개 구성 요소, 축은 기반 개념에 직접 매핑되지 않음
# 개념적으로 유사한 단어들은 임베딩 공간의 인접 영역에 매핑, 임베딩 공간의 축은 임의의 부동소수점 차원

In [ ]:
# 임베딩은 책 내용을 벗어나므로 신경망을 사용하여 임베딩이 만들어질 수 있음과 문장에서 개념상으로 가까운 단어를 예측해낼 수 있다는 것만 이해

In [ ]:
# 임베딩에서 비슷한 단어들끼리 군집하며 일관된 공간 관계를 유지함

In [ ]:
# 현대식 임베딩 모델은 더 정교하고 문맥에 민감함
# 사전의 단어 매핑이 특정 위치로 고정되지 않고 주변을 둘러싼 문장에서 영향을 받음

In [ ]:
# 어휘 집합 내의 많은 개체가 숫자 벡터로 표현되어야 할 경우 임베딩은 필수적이나 이 책에서는 사용하지 않을 예정
# 텍스트를 표현하고 처리하는 방법을 일반적으로 카테고리 데이터를 다루는 예제로도 볼 수 있다고 믿기 때문

In [ ]:
# 텍스트가 아닌 경우 임의의 숫자로 시작해서 학습 문제의 일부로 놓고 발전시켜 가는 표준적인 방법 존재
# 따라서 임베딩은 모든 카테고리 데이터에 대한 원핫 인코딩의 훌륭한 대체재
# 텍스트 다루는 문제에서 사전 학습된 임베딩 개선시켜 가는 것도 일반적인 방법

In [ ]:
# 동시에 발견되는 부분에 관심 있을 시에는 단어 임베딩이 일종의 청사진 역할을 함 Ex) 추천 시스템

## 6절 - 결론

In [ ]:
# 가장 흔한 형태의 데이터를 어떻게 읽어들이는지와 신경망에 넣기 위한 변환에 대해 배움
# 실제로는 하나의 덩어리로 기술하기를 희망하는 경우보다 더 다양한 데이터 포맷이 존재
# 텐서에 익숙해지고 데이터를 텐서에 어떻게 저장하는지도 이해함

## 7절 - 연습 문제

In [7]:
# imageio로 이미지 가져오기
img1_np = imageio.imread('../flower1.png')
img2_np = imageio.imread('../flower2.png')
img3_np = imageio.imread('../flower3.png')

<ipython-input-7-b4d7776e17c7>:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1_np = imageio.imread('../flower1.png')
<ipython-input-7-b4d7776e17c7>:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2_np = imageio.imread('../flower2.png')
<ipython-input-7-b4d7776e17c7>:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img3_np = imageio.imread('../flower3.png')


In [13]:
# 텐서로 변환 후 텐서 전치
img1 = torch.from_numpy(img1_np)
out1 = img1.permute(2, 0, 1)
img2 = torch.from_numpy(img2_np)
out2 = img2.permute(2, 0, 1)
img3 = torch.from_numpy(img3_np)
out3 = img3.permute(2, 0, 1)
print(out1.shape, out1)
print(out2.shape, out2)
print(out3.shape, out3)

torch.Size([3, 183, 275]) tensor([[[201, 201, 200,  ..., 155, 152, 151],
         [201, 201, 200,  ..., 155, 152, 151],
         [201, 201, 200,  ..., 157, 155, 154],
         ...,
         [130, 140, 156,  ..., 125, 116, 104],
         [124, 133, 147,  ..., 119, 111, 100],
         [117, 127, 140,  ..., 114, 107,  98]],

        [[207, 207, 208,  ..., 168, 169, 168],
         [207, 207, 208,  ..., 168, 169, 168],
         [207, 207, 208,  ..., 170, 170, 169],
         ...,
         [120, 131, 147,  ..., 103,  92,  80],
         [115, 127, 141,  ...,  97,  87,  76],
         [111, 123, 136,  ...,  92,  83,  74]],

        [[231, 231, 231,  ..., 187, 189, 188],
         [231, 231, 231,  ..., 187, 189, 188],
         [231, 231, 229,  ..., 189, 191, 190],
         ...,
         [121, 132, 148,  ...,  90,  80,  68],
         [116, 127, 141,  ...,  84,  75,  64],
         [111, 122, 135,  ...,  79,  71,  62]]], dtype=torch.uint8)
torch.Size([3, 187, 269]) tensor([[[242, 242, 242,  ..., 245,

In [46]:
# float형으로 변경 후 dim=0으로 평균값 및 shape 구하기
out1 = out1.float()
print(out1.mean(dim=0))
print(out1.shape, out1.mean(dim=0).shape)
out2 = out2.float()
print(out2.mean(dim=0))
print(out2.shape, out2.mean(dim=0).shape)
out3 = out3.float()
print(out3.mean(dim=0))
print(out3.shape, out3.mean(dim=0).shape)

tensor([[213.0000, 213.0000, 213.0000,  ..., 170.0000, 170.0000, 169.0000],
        [213.0000, 213.0000, 213.0000,  ..., 170.0000, 170.0000, 169.0000],
        [213.0000, 213.0000, 212.3333,  ..., 172.0000, 172.0000, 171.0000],
        ...,
        [123.6667, 134.3333, 150.3333,  ..., 106.0000,  96.0000,  84.0000],
        [118.3333, 129.0000, 143.0000,  ..., 100.0000,  91.0000,  80.0000],
        [113.0000, 124.0000, 137.0000,  ...,  95.0000,  87.0000,  78.0000]])
torch.Size([3, 183, 275]) torch.Size([183, 275])
tensor([[236.0000, 236.0000, 236.0000,  ..., 241.6667, 241.6667, 241.6667],
        [236.0000, 236.0000, 236.0000,  ..., 241.6667, 241.6667, 241.6667],
        [236.0000, 236.0000, 236.0000,  ..., 241.6667, 241.6667, 241.6667],
        ...,
        [234.6667, 234.6667, 234.6667,  ..., 239.6667, 239.6667, 239.6667],
        [234.6667, 234.6667, 234.6667,  ..., 239.6667, 239.6667, 239.6667],
        [234.6667, 234.6667, 234.6667,  ..., 239.6667, 239.6667, 239.6667]])
torch.Size(

In [47]:
# 각 채널 별 평균 구하기
rgb1 = []
rgb2 = []
rgb3 = []
for c in range(3):
    rgb1.append(torch.mean(out1[c,:]))
    rgb2.append(torch.mean(out2[c,:]))
    rgb3.append(torch.mean(out3[c,:]))
print(rgb1, rgb2, rgb3)

[tensor(134.0753), tensor(115.2756), tensor(118.7991)] [tensor(235.6487), tensor(203.3244), tensor(225.2609)] [tensor(59.0326), tensor(74.7687), tensor(47.3091)]


In [48]:
# 이 파일 소스 코드의 모든 단어에 대한 인덱스 dict 이용하여 만들기
with open('../4.7.2.txt', encoding='utf8') as f:
    text = f.read()
line = text.split("\n")

def clean_words(input_str):
    punctuation = '.,;:“!?`”"_-=/\\+#%*&(){}[]'
    word_list = input_str.lower().replace('\n',' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list


word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

print(len(word2index_dict), word2index_dict['word_list'])

226 221


In [51]:
# 인덱스로 원핫 인코딩 만들기
word_t = torch.zeros(len(word2index_dict), len(word2index_dict))
for i, word in enumerate(word2index_dict):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))
print(word_t)

 0    0 
 1    1 '../data/volumetric-dicom'
 2    2 '../data/winequality-white.csv'
 3    3 '.format(i
 4    4 '.png'
 5    5 'dicom'
 6    6 0
 7    7 1
 8    8 1)[:1
 9    9 1.0
10   10 10
11   11 128
12   12 141.83
13   13 1].long
14   14 2
15   15 20
16   16 24
17   17 255.0
18   18 256
19   19 3
20   20 4
21   21 5
22   22 6.2f
23   23 6.2f}'.format(i
24   24 7
25   25 9
26   26 <
27   27 >
28   28 actual_indexes
29   29 actual_indexes.dtype
30   30 actual_indexes.shape
31   31 actual_indexes.sum
32   32 args
33   33 as
34   34 bad_data
35   35 bad_data.shape
36   36 bad_indexes
37   37 bad_indexes.dtype
38   38 bad_indexes.shape
39   39 bad_indexes.sum
40   40 bad_mean
41   41 batch
42   42 batch.float
43   43 batch.shape[1
44   44 batch[i
45   45 batch_size
46   46 bikes
47   47 bikes.shape
48   48 bikes.shape[1
49   49 bikes.stride
50   50 bikes.view(-1
51   51 bikes[:24].long
52   52 bikes_numpy
53   53 c
54   54 clean_words(line
55   55 col_list
56   56 converters={1
57   57 